In [ ]:
import csv
import os
import sklearn
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Train Data Combination of Several Train Collects on Simulator

In [ ]:
#
# Needed one time to concatenate data collected in different runs of the simulator
#
# data_path = "../../tvdata"
# data_log_filename = "driving_log.csv"
# data_fout = data_path + "/" + data_log_filename

# data_dirs = os.listdir(data_path)
# fout = open(data_fout, "a")

# try:
#     for i in range(len(data_dirs)):
#         data_fin = data_path + "/" + data_dirs[i] + "/" + data_log_filename
#         print ("processing: ", data_fin)
#         for line in open(data_fin):
#             fout.write(line)
# except:
#     print("Error in data concatenation...")
#     pass

# fout.close()


## Data Augmentation Experimentation

In [ ]:
# Augmentation of images by flipping images left to right and changing the sign of the steering angle. 
# This was tried out in a seperate workspace and performance did not improve over the 
# driving set collected driving both ways in the simulator.

In [ ]:
train_and_validate_samples = []

with open('../../tvdata/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        train_and_validate_samples.append(line)


## Training and Validation Data Stats and Visualization

In [ ]:
# Each training sample has 3 images
print ("Total training samples: ", len(train_and_validate_samples) * 3)

meas = []
iter = 0
# Visualize the steering samples.
for line in train_and_validate_samples:      
    steering_meas = float(line[3])
    meas.append(steering_meas)
    iter += 1
    
    # plot some image samples
    if ((iter % 2500) == 0):
        print ("Steering Meas: ", steering_meas)
        
        file_pathname = line[0].split("/")[-1]
        image = plt.imread("../../tvdata/IMG/" + file_pathname)
        plt.imshow(image)
        plt.show()
        
        file_pathname = line[1].split("/")[-1]
        image = plt.imread("../../tvdata/IMG/" + file_pathname)
        plt.imshow(image)
        plt.show()
        
        file_pathname = line[2].split("/")[-1]
        image = plt.imread("../../tvdata/IMG/" + file_pathname)
        plt.imshow(image)
        plt.show()
        
        
    
plt.plot(meas)
plt.xlabel("samples")
plt.ylabel("steering angle")
plt.title("Steering angle visualization")
plt.show()
plt.plot(np.diff(meas))
plt.xlabel("samples")
plt.ylabel("steering angle derivative")
plt.title("Steering angle derivative visualization")

In [ ]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(train_and_validate_samples, test_size=0.3)

In [ ]:
# Generator
correction = 0.1
used_batch_size = 128
data_dir = "../../tvdata/IMG/"

def generator(samples, base_path, batch_size=used_batch_size):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurements = []
            for line in batch_samples:
                # Center Image.
                file_pathname = line[0].split("/")[-1]
                image = plt.imread(base_path + file_pathname)
                images.append(image)
                steering_meas = float(line[3])
                measurements.append(steering_meas)

                # Apply + correction to steering angle of left image
                l_file_pathname = line[1].split("/")[-1]
                image = plt.imread(base_path + l_file_pathname.lstrip())
                images.append(image)
                measurements.append(steering_meas + correction)
    
                # Apply - correction to steering angle of right image.
                r_file_pathname = line[2].split("/")[-1]
                image = plt.imread(base_path + r_file_pathname.lstrip())
                images.append(image)
                measurements.append(steering_meas - correction)

            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

In [ ]:
# compile and train the model using the generator function
train_generator = generator(train_samples, data_dir, batch_size=used_batch_size)
validation_generator = generator(validation_samples, data_dir, batch_size=used_batch_size)

In [ ]:
def show_loss_stats(history_object):
    ### plot the training and validation loss for each epoch
    plt.plot(history_object.history['loss'])
    plt.plot(history_object.history['val_loss'])
    plt.title('model mean squared error loss')
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    plt.legend(['training set', 'validation set'], loc='upper right')
    plt.show()

In [ ]:
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda, Conv2D, MaxPooling2D, Cropping2D, Dropout
from keras import optimizers
print (keras.__version__)

def lenet_model():
    print ('\nTraining using lenet_model()...\n')
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D( cropping=((50,20), (0,0))) )
    model.add(Conv2D(6, (5,5) ,activation="relu"))
    model.add(MaxPooling2D())
    model.add(Conv2D(6, (5,5) ,activation="relu"))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dense(84))
    model.add(Dense(1))

    model.compile(loss="mse", optimizer="adam")
    model.summary()
    
#    history_object = model.fit(X_train,y_train,validation_split=0.2, shuffle=True, epochs=1, batch_size=512, verbose=2)
    history_object = model.fit_generator(train_generator, steps_per_epoch=math.ceil( len(train_samples)/used_batch_size ),
                                         validation_data=validation_generator,
                                         validation_steps=math.ceil((len(validation_samples)/used_batch_size)), 
                                         epochs=15,verbose=2,initial_epoch=0)
    model.save('model.h5')
    show_loss_stats(history_object)

def regularized_nvidia_model():
    print('\nTraining using Regularized Nvidia model...\n')
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D( cropping=((50,20), (0,0))) )
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(48, (5,5), strides=(2,2), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    
    history_object = model.fit_generator(train_generator, steps_per_epoch=math.ceil( len(train_samples)/used_batch_size ),
                                         validation_data=validation_generator,
                                         validation_steps=math.ceil((len(validation_samples)/used_batch_size)), 
                                         epochs=15,verbose=2,initial_epoch=0)
    
#    opt = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
#    opt = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
#    model.compile(loss="mse", optimizer=opt)
#    history_object = model.fit(X_train,y_train,validation_split=0.3, shuffle=True, epochs=8, batch_size=512, verbose=2)

    model.save('model.h5')
    show_loss_stats(history_object)

In [ ]:
#lenet_model()    
regularized_nvidia_model()